In [227]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime
from datetime import date
import glob

from scipy.optimize import root_scalar
import scipy.optimize as opt

In [202]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [211]:
def find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file

def clean_position(position):
    position_copy = position.copy()
    position_copy = position_copy[
        position_copy["Current Value"].notna()
    ]  # remove rows without current value
    position_copy["Current Value"] = transfer_dollar_to_float(
        position_copy["Current Value"]
    )
    position_copy["Cost Basis Total"] = transfer_dollar_to_float(
        position_copy["Cost Basis Total"]
    )
    return position_copy

def transfer_dollar_to_float(dat):
    """
    Change "$123,456" to 123455
    """
    return dat.str.replace("$", "", regex=False).astype(float)


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)

    transactions = combine_transaction_files(transaction_files)
    transactions = clean_transactions(transactions)
    return transactions

def combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(13)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions


def clean_transactions(transactions):
    transactions_copy = transactions.copy()
    transactions_copy = transactions_copy[
        transactions_copy["Amount ($)"].notna()
    ]  # remove rows without  value
    transactions_copy["Run Date"] = pd.to_datetime(
        transactions_copy["Run Date"], format=" %m/%d/%Y"
    ).dt.date
    transactions_copy["Settlement Date"] = pd.to_datetime(
        transactions_copy["Settlement Date"], format="%m/%d/%Y"
    ).dt.date
    transactions_copy.loc[transactions_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    transactions_copy["Symbol"] = transactions_copy[
        "Symbol"
    ].str.lstrip()  # remove space at the beginning of Symbol
    transactions_copy = transactions_copy.sort_values(by="Run Date").reset_index(
        drop=True
    )
    return transactions_copy

In [212]:
## load position
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'


In [213]:
position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
position_files = glob.glob(position_file_path_pattern)
position_file = find_latest_position_file(position_files)
position = pd.read_csv(position_file)
position = clean_position(position)

In [214]:
transaction_file_path_pattern = os.path.join(
    data_folder_path, transaction_file_pattern
)
transaction_files = glob.glob(transaction_file_path_pattern)

transactions = combine_transaction_files(transaction_files)
transactions = clean_transactions(transactions)
print(f"The latest transaction date is {transactions['Run Date'].max()}")

The latest transaction date is 2025-03-06


In [ ]:
class Portfolio:
    
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        
        self.today = date.today()
        self.individual_transactions = self.transactions[
            self.transactions["Account"].isin(["Individual Z23390746","Individual"]) 
        ]
        self.pension_transactions = self.transactions[
            self.transactions["Account"].isin(["ERNST & YOUNG 401(K) 86964","ERNST & YOUNG 401(K)"]) 
        ]
        self.HSA_transactions = self.transactions[
            self.transactions["Account"].isin(["Health Savings Account 241802439","Health Savings Account"]) 
        ]
        self.cash_transactions = self.transactions[
            self.transactions["Account"].isin(["Cash Management (Individual) Z06872898","Cash Management (Individual)"])
        ]
        
        self.individual_position = self.position[
            self.position["Account Number"].isin(["Z23390746"])
        ]
        self.pension_position = self.position[
            self.position["Account Number"].isin(["86964"])
        ]
        self.HSA_position = self.position[
            self.position["Account Number"].isin(["241802439"])
        ]
        self.cash_position = self.position[
            self.position["Account Number"].isin(["Z06872898"])
        ]
        
    def show_individual_investment_distribution(self):
        individual_investment_distribution = self.get_individual_investment_distribution()
        individual_investment_distribution["Percent"] = [
            f"{x * 100:.2f}%" for x in individual_investment_distribution["Percent"]
        ]
        print(pd.DataFrame(individual_investment_distribution))
        
    def get_individual_investment_distribution(self):
        total_mv = self.get_individual_mv()
        stock_mv = self.get_stock_mv()
        bill_mv = self.get_bill_mv()
        cash_mv = self.get_cash_mv()
        result = {
            "Class": ["stock", "bill", "cash", "total"],
            "Amount": [
                stock_mv,
                bill_mv,
                cash_mv,
                total_mv,
            ],
            "Percent": [
                stock_mv / total_mv,
                bill_mv / total_mv,
                cash_mv / total_mv,
                total_mv / total_mv,
            ],
        }

        return result
    
    def get_total_investment(self):
        total_investment = self.individual_transactions[
            self.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"].sum()
        return total_investment
    
    
    def get_cash_mv(self):
        cash_index = self.individual_position['Description']=='HELD IN MONEY MARKET'
        return self.individual_position[cash_index]['Current Value'].sum()
    
    def get_bill_mv(self):
        bill_index = self.individual_position['Description'].str.contains("UNITED STATES TREAS BILLS")
        return self.individual_position[bill_index]['Current Value'].sum()
    
    def get_stock_position(self):
        cash_index = self.individual_position['Description']=='HELD IN MONEY MARKET'
        bill_index = self.individual_position['Description'].str.contains("UNITED STATES TREAS BILLS")
        return self.individual_position[(~cash_index)&(~bill_index)]
    
    def get_stock_mv(self):
        stock_position = self.get_stock_position()
        return stock_position['Current Value'].sum()
    
    def get_individual_mv(self):
        return self.individual_position['Current Value'].sum()
    
    def get_transaction_for_irr_calculation(self):
        stock_transaction = self.get_stock_transaction()
        negative_stock_position = self.get_negative_stock_mv()
        stock_transaction_with_negative_position = pd.concat([stock_transaction,negative_stock_position],ignore_index=True)
        stock_transaction_with_negative_position['time_diffs_in_year']= (
                self.today - stock_transaction_with_negative_position.loc[:, "Run Date"]
            ).apply(lambda x: x.days) / 365.25
        return stock_transaction_with_negative_position
        
    
    def get_stock_transaction(self):
        bill_transaction_index = self.individual_transactions['Symbol'].str.startswith('912')
        return self.individual_transactions[~bill_transaction_index]
    
    def get_negative_stock_mv(self):
        stock_mv = self.get_stock_position()[['Symbol','Current Value']]
        total_investment_row = pd.DataFrame({
            'Symbol':['Transfer'],
            'Current Value': self.get_individual_mv()
        })
        stock_mv = pd.concat([stock_mv,total_investment_row],ignore_index=True)
        stock_mv['Current Value'] = -stock_mv['Current Value']
        stock_mv = stock_mv.rename(columns={'Current Value':'Amount ($)'})
        stock_mv['Run Date'] = date.today()
        return stock_mv
    
    @staticmethod
    
    def npv(rate, transactions):
        return np.sum(
            transactions["Amount ($)"]
            * (1 + rate) ** transactions["time_diffs_in_year"]
        )
            
    def calculate_irr(transactions, npv, guess=0.1):
        irr = opt.newton(lambda rate:npv(rate, transactions),guess)
        return irr

In [221]:
my_prot = Portfolio(transactions,position)

In [ ]:
temp = my_prot.get_transaction_for_irr_calculation()
temp['Run Date'][200]

datetime.date(2025, 2, 13)

In [225]:
temp = temp[temp['Symbol']=='Transfer']
temp

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Account Number,time_diffs_in_year
0,2022-07-26,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,1000.0,NaT,NaN,2.614648
7,2022-08-08,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,29000.0,NaT,NaN,2.579055
73,2023-12-18,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,30000.0,NaT,NaN,1.218344
77,2024-01-22,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,40000.0,NaT,NaN,1.122519
90,2024-03-27,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,1500.0,NaT,NaN,0.944559
97,2024-04-15,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,148500.0,NaT,NaN,0.892539
98,2024-04-16,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,-148500.0,NaT,NaN,0.889802
99,2024-04-17,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,148500.0,NaT,NaN,0.887064
100,2024-04-29,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,100000.0,NaT,NaN,0.854209
104,2024-04-30,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.0,NaN,NaN,NaN,NaN,-100000.0,NaT,NaN,0.851472


In [226]:
def npv(rate, transactions):
    return np.sum(
        transactions["Amount ($)"]
        * (1 + rate) ** transactions["time_diffs_in_year"]
    )

0.07356895734712868

In [116]:
my_prot.get_stock_transaction()['Run Date']

0     2022-07-26
1     2022-07-29
2     2022-07-29
3     2022-08-05
4     2022-08-05
         ...    
447   2025-02-28
449   2025-02-28
450   2025-02-28
452   2025-03-03
453   2025-03-04
Name: Run Date, Length: 207, dtype: datetime64[ns]

In [103]:
my_prot.get_stock_transaction()

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Account Number
0,2022-07-26,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.00,NaN,NaN,NaN,NaN,1000.00,NaT,NaN
1,2022-07-29,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.00,NaN,NaN,NaN,-0.22,NaT,NaN
2,2022-07-29,Individual Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT,NaN
3,2022-08-05,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.00,164.61,NaN,NaN,NaN,-1481.45,2022-08-09,NaN
4,2022-08-05,Individual Z23390746,YOU BOUGHT STARBUCKS CORP COM USD0.001 (SBUX)...,SBUX,STARBUCKS CORP COM USD0.001,Cash,1.00,85.20,NaN,NaN,NaN,-85.20,2022-08-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2025-02-28,Individual,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,186.16,NaT,Z23390746
449,2025-02-28,Individual,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,186.16,1.00,NaN,NaN,NaN,-186.16,NaT,Z23390746
450,2025-02-28,Individual,DIVIDEND RECEIVED STARBUCKS CORP COM USD0.001...,SBUX,STARBUCKS CORP COM USD0.001,Cash,0.00,NaN,NaN,NaN,NaN,18.30,NaT,Z23390746
452,2025-03-03,Individual,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.00,NaN,NaN,NaN,NaN,25500.00,NaT,Z23390746


In [ ]:
my_prot.individual_transactions[~my_prot.individual_transactions['Symbol'].str.startswith('912')]

array(['Transfer', 'FZFXX', 'AAPL', 'SBUX', 'JPM', 'AXP', 'GOOGL', 'AMZN',
       'NKE', 'TSLA', 'FXAIX', 'MCD', 'FSKAX', 'FSPSX', 'COKE', 'MSFT',
       'BRKB', 'KHC', 'VZ', 'NVDA', 'PEP'], dtype=object)